# Route53 도메인 등록과 활용

## Demo - Amazon Route53 도메인 등록
- Part 1
    - Route53에서 도메인을 등록, Route53에서 관리하도록 설정
    - 이후 다양한 레코드를 생성해서 AWS 리소스와 연결 (EC2, S3, ALB)
- Part 2
    - 외부에서 도메인 구입 후 Route53에서 생성한 Hosted Zone의 NS 서버로 변경
        - KR 등 Route53에서 구매할 수 없는 경우 / 이미 도메인을 보유했을 경우
- 둘 다 비용 발생
    - 도메인 구입에 비용이 발생 (Route53에서 등록하더라도 크레딧 사용 불가능)
    - Route53은 프리티어가 없기에 매달 Hosted Zone당 0.5달러 + 쿼리 비용 발생

## 실습 순서
1. Route53 콘솔 이동 (글로벌 서비스라 리전 선택 불가능)
    - 등록된 도메인 : 도메인 등록 (한참 기다려야한다...)
2. EC2 인스턴스를 연결해보자
    - 키페어 없이
    - 보안그룹 디폴트
    - 유저데이터에 스크립트 입력
3. Route53 콘솔로 이동
    - 호스팅 영역 : 레코드 생성
        - 레코드 이름 : web
        - 값 : 인스턴스의 퍼블릭 IP
        - TTL : 60
        - 레코드 유형 : A
    - 레코드를 하나 더 생성해보자
        - 레코드 이름 : webdns
        - 레코드 유형 : CNAME
        - 값 : 인스턴스의 퍼블릭 IPv4 dns 
        - TTL : 60
4. S3 스태틱 호스팅을 연결해보자
    - 버킷 생성 : 퍼블릭 엑세스 허용
    - 권한 설정 : 버킷 정책 편집 (스크립트 복붙)
    - 속성 : 정적 웹 사이트 호스팅 활성화, index.html 입력 및 업로드
    - Route53에서 CNAME 레코르를 통해 연결해보자
        - 404 Not Found가 뜰거임 : 버킷을 사용하여 정적 호스팅을 할 때는 버킷명과 레코드명이 동일해야함
        - 버킷 이름을 도메인 이름과 같게 만들자 (레코드도 새로 생성)
5. Apex 도메인에 정적 호스팅을 연결해보자
    - 버킷 생성 : Apex 도메인과 같은 이름
        - 권한 설정, 속성 편집
    - 레코드 생성 : 레코드 이름 공란, CNAME
        - APEX 도메인을 사용할 수 없다는 에러가 뜸
        - ALIAS를 사용해야함 : A 레코드 사용
            - 트래픽 라우팅 대상 : S3
6. ALB를 Route53에 연결해보자
    - 대상 그룹 생성
        - 상태검사 : index.html
        - 만들었던 인스턴스를 보류 대상으로 포함
    - 로드밸런서 생성
        - ALB
        - 가용역역 모두 선택
        - 보안그룹 디폴트
        - 대상그룹 선택
    - 레코드 생성
        - 레코드 유형 : A
        - ALIAS 사용 : ALB선택
7. 리소스 정리 : ALB, 인스턴스